In [1]:
import pandas as pd
from sklearn.neural_network import MLPClassifier
import VisualizeNN as VisNN
import piplite
await piplite.install("seaborn")
import pyodide
import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline
sns.set(rc={'figure.figsize':(11.7,8.27)})